# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [6]:
# links = fetch_website_links("https://edwarddonner.com")
links = fetch_website_links("https://aclanthology.org/people/juan-carlos-martinez-santos/")
links

['https://aclanthology.org/',
 '/posts/',
 '/faq/',
 '/info/corrections/',
 '/info/contrib/',
 'https://github.com/acl-org/acl-anthology/',
 'https://aclanthology.org/2025.semeval-1.167.pdf',
 '/2025.semeval-1.167.bib',
 '#abstract-2025--semeval-1--167',
 '/2025.semeval-1.167/',
 '/people/daniel-pena-gnecco/',
 '/people/juan-carlos-martinez-santos/',
 '/people/edwin-puertas/',
 '/volumes/2025.semeval-1/',
 'https://aclanthology.org/2025.semeval-1.202.pdf',
 '/2025.semeval-1.202.bib',
 '#abstract-2025--semeval-1--202',
 '/2025.semeval-1.202/',
 '/people/anderson-morillo/',
 '/people/edwin-puertas/',
 '/people/juan-carlos-martinez-santos/',
 '/volumes/2025.semeval-1/',
 'https://aclanthology.org/2024.semeval-1.193.pdf',
 '/2024.semeval-1.193.bib',
 '#abstract-2024--semeval-1--193',
 'https://aclanthology.org/attachments/2024.semeval-1.193.SupplementaryMaterial.zip',
 'https://aclanthology.org/attachments/2024.semeval-1.193.SupplementaryMaterial.txt',
 'https://aclanthology.org/2024.semev

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [10]:
print(get_links_user_prompt("https://aclanthology.org/people/juan-carlos-martinez-santos/"))


Here is the list of links on the website https://aclanthology.org/people/juan-carlos-martinez-santos/ -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://aclanthology.org/
/posts/
/faq/
/info/corrections/
/info/contrib/
https://github.com/acl-org/acl-anthology/
https://aclanthology.org/2025.semeval-1.167.pdf
/2025.semeval-1.167.bib
#abstract-2025--semeval-1--167
/2025.semeval-1.167/
/people/daniel-pena-gnecco/
/people/juan-carlos-martinez-santos/
/people/edwin-puertas/
/volumes/2025.semeval-1/
https://aclanthology.org/2025.semeval-1.202.pdf
/2025.semeval-1.202.bib
#abstract-2025--semeval-1--202
/2025.semeval-1.202/
/people/anderson-morillo/
/people/edwin-puertas/
/people/juan-carlos-martinez-santos/
/volumes/2025.semeval-1/
https://aclanthology.org/2024.semeval-1.193.pdf
/2024.semeval-1.193.bib
#abst

In [11]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [12]:
select_relevant_links("https://aclanthology.org/people/juan-carlos-martinez-santos/")

{'links': [{'type': 'home page', 'url': 'https://aclanthology.org/'},
  {'type': 'source code', 'url': 'https://github.com/acl-org/acl-anthology/'},
  {'type': 'contributors information',
   'url': 'https://aclanthology.org/info/contrib/'},
  {'type': 'credits page', 'url': 'https://aclanthology.org/info/credits/'}]}

In [13]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [14]:
select_relevant_links("https://aclanthology.org/people/juan-carlos-martinez-santos/")

Selecting relevant links for https://aclanthology.org/people/juan-carlos-martinez-santos/ by calling gpt-5-nano
Found 2 relevant links


{'links': [{'type': 'organization homepage',
   'url': 'https://aclanthology.org/'},
  {'type': 'source code repository',
   'url': 'https://github.com/acl-org/acl-anthology/'}]}

In [15]:
select_relevant_links("https://aclanthology.org/people/juan-carlos-martinez-santos/")

Selecting relevant links for https://aclanthology.org/people/juan-carlos-martinez-santos/ by calling gpt-5-nano
Found 2 relevant links


{'links': [{'type': 'company homepage', 'url': 'https://aclanthology.org/'},
  {'type': 'open-source repository',
   'url': 'https://github.com/acl-org/acl-anthology/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [16]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [18]:
print(fetch_page_and_all_relevant_links("https://aclanthology.org/people/juan-carlos-martinez-santos/"))

Selecting relevant links for https://aclanthology.org/people/juan-carlos-martinez-santos/ by calling gpt-5-nano
Found 4 relevant links
## Landing Page:

Juan Carlos Martinez Santos - ACL Anthology

ACL Anthology
News
(current)
FAQ
(current)
Corrections
(current)
Submissions
(current)
Github
Juan Carlos
Martinez Santos
2025
pdf
bib
abs
V
erba
N
ex
AI
at
S
em
E
val-2025 Task 2: Enhancing Entity-Aware Translation with
W
ikidata-Enriched
M
arian
MT
Daniel Peña Gnecco
|
Juan Carlos Martinez Santos
|
Edwin Puertas
Proceedings of the 19th International Workshop on Semantic Evaluation (SemEval-2025)
This paper presents the VerbaNexAi Lab system for SemEval-2025 Task 2: Entity-Aware Machine Translation (EA-MT), focusing on translating named entities from English to Spanish across categories such as musical works, foods, and landmarks. Our approach integrates detailed data preprocessing, enrichment with 240,432 Wikidata entity pairs, and fine-tuning of the MarianMT model to enhance entity transl

In [19]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [20]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [21]:
get_brochure_user_prompt("Juan Carlos Martinez Santos", "https://aclanthology.org/people/juan-carlos-martinez-santos/")

Selecting relevant links for https://aclanthology.org/people/juan-carlos-martinez-santos/ by calling gpt-5-nano
Found 9 relevant links


'\nYou are looking at a company called: Juan Carlos Martinez Santos\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nJuan Carlos Martinez Santos - ACL Anthology\n\nACL Anthology\nNews\n(current)\nFAQ\n(current)\nCorrections\n(current)\nSubmissions\n(current)\nGithub\nJuan Carlos\nMartinez Santos\n2025\npdf\nbib\nabs\nV\nerba\nN\nex\nAI\nat\nS\nem\nE\nval-2025 Task 2: Enhancing Entity-Aware Translation with\nW\nikidata-Enriched\nM\narian\nMT\nDaniel Peña Gnecco\n|\nJuan Carlos Martinez Santos\n|\nEdwin Puertas\nProceedings of the 19th International Workshop on Semantic Evaluation (SemEval-2025)\nThis paper presents the VerbaNexAi Lab system for SemEval-2025 Task 2: Entity-Aware Machine Translation (EA-MT), focusing on translating named entities from English to Spanish across categories such as musical works, foods, and landmarks. Our approach integ

In [22]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [23]:
create_brochure("Juan Carlos Martinez Santos", "https://aclanthology.org/people/juan-carlos-martinez-santos/")

Selecting relevant links for https://aclanthology.org/people/juan-carlos-martinez-santos/ by calling gpt-5-nano
Found 2 relevant links


# Brochure: Juan Carlos Martinez Santos – Expert in Computational Linguistics and NLP Research

---

## About Juan Carlos Martinez Santos

Juan Carlos Martinez Santos is a distinguished researcher specializing in computational linguistics and natural language processing (NLP). He actively contributes to state-of-the-art advancements in machine translation, entity-aware NLP tasks, and AI-driven language evaluations. His work is showcased in reputable venues such as the International Workshop on Semantic Evaluation (SemEval).

---

## Key Research Contributions

- **Entity-Aware Machine Translation (EA-MT)**
  - Developed the VerbaNexAi Lab system for SemEval-2025 Task 2.
  - Enhanced translation of named entities (e.g., musical works, foods, landmarks) from English to Spanish by enriching training data with over 240,000 Wikidata entity pairs using MarianMT, a neural machine translation model.
  - Achieved a high fluency score (COMET 87.09) demonstrating strengths in natural, fluent translations, though challenges remain in precise entity translation accuracy compared to dynamic large language model (LLM) approaches.

- **Fact Retrieval and Hallucination Detection in LLMs**
  - Pioneered methods at SemEval-2025 Task 3 to improve hallucination detection in language models using:
    - Chain-of-thought prompting with Google snippet context retrieval.
    - Traditional NLP semantic ranking and token-level extraction techniques.
  - These innovations contribute substantially to improving information accuracy and trustworthiness in LLM outputs.

- **Emotional Causality in Conversations**
  - Investigated the detection of emotional causality in conversational AI, advancing understanding of nuanced linguistic phenomena important for empathetic and context-aware AI systems.

---

## The VerbaNexAi Lab

This research lab, led in part by Juan Carlos Martinez Santos, focuses on:

- Building cutting-edge NLP systems centered on lexical and semantic enrichment.
- Leveraging external knowledge bases (like Wikidata) for enhanced language understanding.
- Bridging symbolic and neural methods to improve model explainability and precision.

---

## Company Culture & Values

- **Innovation-Driven:** Committed to advancing natural language understanding through data-intensive, creative research.
- **Collaborative Environment:** Valuing interdisciplinary teamwork among linguists, data scientists, and AI engineers.
- **Transparency and Open Science:** Frequently publishes in open venues, contributes to shared repositories, and engages with the global NLP community.
- **Focus on Real-World Impact:** Strives to create robust AI systems that address challenges in machine translation, information retrieval, and natural language generation.

---

## Customers and Audience

Juan Carlos Martinez Santos primarily serves:

- **Academic and Research Institutions** focused on computational linguistics.
- **AI and Tech Companies** seeking expertise in enhancing machine translation and LLM accuracy.
- **NLP Developers** interested in code and data from open repositories like the ACL Anthology and GitHub.

---

## Career and Collaboration Opportunities

- **Research Positions:** Opportunities for computational linguistics researchers to join ongoing projects in entity-aware translation and fact verification.
- **Internships and Fellowships:** For graduate students passionate about semantic evaluation, machine translation, and NLP tool development.
- **Open Source Contributions:** Active encouragement for developers and researchers worldwide to contribute to open-source projects hosted on GitHub (e.g., acl-org/acl-anthology).

---

## Learn More & Get Involved

- **Publications & Papers:** Access the latest research papers through the ACL Anthology repository.
- **GitHub Repository:** Explore data and software for building and extending the ACL Anthology.
- **Community Events:** Participate in workshops such as SemEval and other ACL-affiliated conferences.

---

**Contact & Follow**

- Stay updated with ongoing advancements in NLP and language technologies by following Juan Carlos Martinez Santos’ contributions in the ACL Anthology and related GitHub projects.

---

Empowering language technologies — bringing enriched, accurate, and fluent AI communication to the world.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [24]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [25]:
stream_brochure("Juan Carlos Martinez Santos", "https://aclanthology.org/people/juan-carlos-martinez-santos/")

Selecting relevant links for https://aclanthology.org/people/juan-carlos-martinez-santos/ by calling gpt-5-nano
Found 2 relevant links


# Brochure: Juan Carlos Martinez Santos & ACL Anthology

---

## About Juan Carlos Martinez Santos

Juan Carlos Martinez Santos is a leading researcher and contributor in the field of computational linguistics and natural language processing (NLP). His recent work focuses on enhancing machine translation and fact retrieval using advanced AI and NLP techniques. His projects emphasize entity-aware translation and hallucination detection in large language models (LLMs), contributing to top-tier international workshops like SemEval.

---

## Overview of ACL Anthology

The **ACL Anthology** is a significant academic repository and collaborative platform hosting over 118,000 papers dedicated to computational linguistics and NLP research. It serves the global community by providing open access to cutting-edge research outputs, including numerous proceedings, workshops, and conference papers.

- Hosted papers cover a diverse range of NLP topics including machine translation, semantic evaluation, emotional causality, and more.
- A diverse catalog spanning decades, from foundational works in the 1980s to the latest breakthroughs from 2025.
- Facilitates research dissemination through several channels: mailing lists, RSS news feeds, and community feedback.
- Supports a thriving ecosystem with dedicated pages for FAQs, news, corrections, and submissions.

---

## Recent Key Research Contributions by Juan Carlos Martinez Santos

### 1. Enhancing Entity-Aware Translation with Wikidata-Enriched MarianMT (SemEval-2025)
- Developed the VerbaNexAi Lab system targeting named-entity rich translation tasks from English to Spanish.
- Integrated a Wikidata corpus with over 240,000 entity pairs to improve translation precision for specific categories like music, food, and landmarks.
- Achieved high fluency scores but identified opportunities to improve precision, especially compared to dynamic LLM approaches.

### 2. Fact Retrieval with Google Snippets for LLM Context Filtering (SemEval-2025)
- Pioneered a hybrid approach combining advanced chain-of-thought prompting with traditional NLP for hallucination detection in large language models.
- Demonstrated superior performance in filtering and fact verification by leveraging Google snippet context extraction alongside semantic ranking.

---

## Company Culture & Values

While ACL Anthology itself is an academic initiative rather than a traditional company, the culture around contributors like Juan Carlos Martinez Santos and the wider ACL community is characterized by:

- **Innovation in AI and NLP:** Constant pursuit of advancing language technologies and sharing breakthroughs globally.
- **Collaboration and Openness:** Promoting open access to research and encouraging multidisciplinary cooperation.
- **Rigorous Research Standards:** Emphasis on peer-reviewed publications, reproducibility, and transparency.
- **Community Engagement:** Hosting events, workshops, and maintaining forums for global NLP scholars and practitioners.

---

## Audience & Customers

- **Academic Researchers & Students:** Access to comprehensive NLP research literature.
- **Industry Practitioners:** Utilizing ACL Anthology as a resource for state-of-the-art methodologies in NLP and AI applications.
- **Event Organizers & Publishers:** Support for conferences and workshops in computational linguistics.
- **Funding & Research Institutions:** Insight into trends and advancements guiding strategic investments in language technologies.

---

## Careers & Opportunities

Though ACL Anthology is a collaborative platform rather than a commercial entity, the associated ecosystem offers opportunities for:

- **Research Positions:** In universities and research labs specializing in computational linguistics and AI.
- **Data Science & NLP Engineering:** Roles in companies applying NLP research to real-world applications, e.g., machine translation, conversational AI.
- **Open Source Collaboration:** Contribution to projects like the ACL Anthology codebase on GitHub, involved in managing data and software essential to the community.

Interested candidates and contributors are encouraged to engage with the community through conferences, workshops like SemEval, and open-source collaborations hosted on GitHub.

---

## Additional Resources

- **ACL Anthology Homepage:** A gateway to a vast archive of ACL research papers and tools.
- **GitHub Repository:** Open-source data and software essential to maintaining and building the ACL Anthology infrastructure.
- **Mailing List & RSS Feeds:** Stay updated on the latest research papers and community announcements.

---

## Contact & Follow

- Visit the [ACL Anthology](https://aclanthology.org) for accessing research and updates.
- Explore the code and contribute at [ACL Anthology GitHub](https://github.com/acl-org/acl-anthology).

---

**Empowering the future of language technology through collaboration, research, and innovation!**

In [26]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("Juan Carlos Martinez Santos", "https://aclanthology.org/people/juan-carlos-martinez-santos/")

Selecting relevant links for https://aclanthology.org/people/juan-carlos-martinez-santos/ by calling gpt-5-nano
Found 3 relevant links


# Brochure: Juan Carlos Martinez Santos – ACL Anthology Contributor & Researcher

---

## About Juan Carlos Martinez Santos

Juan Carlos Martinez Santos is a distinguished researcher active within the computational linguistics and natural language processing (NLP) community, prominently contributing to major international workshops such as SemEval-2025. His work focuses on advanced NLP challenges like Entity-Aware Machine Translation, fact retrieval, and hallucination detection in language models, often integrating current resources such as Wikidata and leveraging the latest AI architectures like MarianMT and large language models (LLMs).

---

## Research Contributions

- **Entity-Aware Machine Translation**  
  Santos co-developed the VerbaNexAi Lab translation system dedicated to enhancing the translation of named entities from English to Spanish within diverse categories (musical works, foods, landmarks). His method enriches data preprocessing through the utilization of over 240,000 Wikidata entity pairs and fine-tunes state-of-the-art MarianMT models to improve translation fluency and accuracy.

- **Fact Retrieval & Hallucination Detection**  
  Engaging with innovative LLM approaches, he helped devise chain-of-thought prompting strategies combined with Google snippets to improve context filtering and reduce hallucinations in machine-generated content. Parallelly, traditional NLP techniques like semantic ranking and token-level extraction are also applied to ensure information reliability.

- **Emotion Causality in Conversations**  
  His recent work in SemEval-2024 explores deciphering emotional causality, signifying his broad interest in nuanced language understanding beyond simple translation.

---

## Company & Community Presence

Juan Carlos Martinez Santos is associated with the ACL Anthology — a premier digital archive hosting over 118,000 papers on computational linguistics and NLP. Through this platform, he shares his cutting-edge research with a global audience including academics, industry experts, and AI practitioners.

The ACL Anthology serves as a focal hub for the NLP community, providing:

- Access to decades of scholarly papers  
- News, events, and calls for submissions  
- A collaborative environment enriched by contributions from leading researchers worldwide  
- Integration with open source resources and code repositories on platforms like GitHub

---

## Company Culture & Values

While Juan Carlos Martinez Santos operates within academic and research frameworks rather than a traditional company, the ethos surrounding his work fosters:

- **Innovation:** Embracing cutting-edge AI, data enrichment, and hybrid methodologies  
- **Collaboration:** Engaging multi-disciplinary teams and global communities via ACL Anthology  
- **Transparency:** Publishing research openly for peer review and continuous improvement  
- **Education:** Supporting the dissemination of knowledge through freely accessible NLP documentation and datasets

---

## Customers & Stakeholders

- **Academic Researchers:** Utilizing his research to advance machine translation, NLP algorithms, and linguistic analysis  
- **AI Industry Practitioners:** Adopting proven methodologies and data practices within commercial translation and chatbot applications  
- **Technology Developers:** Leveraging open source code and pre-trained models for developing intelligent systems  
- **Students & Educators:** Accessing educational resources and research breakthroughs through ACL Anthology archives  

---

## Careers & Opportunities

While Juan Carlos Martinez Santos’s primary involvement is research-focused, opportunities related to his expertise can be found within:

- Research positions at universities and AI labs specializing in NLP  
- Roles in AI companies developing multilingual translation systems and fact-checking tools  
- Collaborative projects affiliated with ACL workshops and conferences  
- Contributions to open-source NLP projects hosted on GitHub and similar platforms

Professionals passionate about language technologies, machine learning, and computational linguistics will find rewarding challenges aligned with Juan Carlos’s research themes.

---

## Connect & Learn More

- **Access Research Papers:** Visit the ACL Anthology to explore Juan Carlos Martinez Santos’s published works on machine translation, fact retrieval, and semantic evaluation.  
- **Follow Ongoing Projects:** Discover code repositories and datasets supporting reproducible research on GitHub via the ACL organizational pages.  
- **Join the NLP Community:** Subscribe to ACL Anthology mailing lists and RSS feeds for the latest updates and event announcements in computational linguistics.

---

**Discover the future of language AI with Juan Carlos Martinez Santos — pushing the boundaries of intelligent translation and fact-aware natural language processing.**

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>